In [ ]:
COMPONENT='seldon-python-base'
FOSSA_DEPENDENCIES = f"../dependencies.{COMPONENT}.yaml"
OUTPUT_FILE = f"../foss.usage.{COMPONENT}.yaml"

In [ ]:
print (f"FOSSA_DEPENDENCIES: {FOSSA_DEPENDENCIES}")
print (f"OUTPUT_FILE: {OUTPUT_FILE}")


In [ ]:
import yaml 

with open("obligations_text.yaml") as f:
    license_dict = yaml.load(f, Loader=yaml.FullLoader)

license_dict


In [ ]:
! pip install pandas

In [ ]:
import yaml, pandas

In [ ]:
with open(FOSSA_DEPENDENCIES, "r") as f:
    dep_dict = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
flat_dict = {}
invalid_entries = []
for entry in dep_dict['dependencies']:
    if entry['bazaar']['register'] == "no":
        primID=entry['bazaar']['prim']
        foss_product_number=primID.split("/")[1]
        nested_dict = {}
        nested_dict["ID"] = entry["ID"]
        nested_dict["usage"] = entry['bazaar']['component_highlevel_description']
        nested_dict["obligation"] = "unknown"
        nested_dict["license"] = "unknown"
        nested_dict["primID"] = primID
        if 'selected_licenses' in entry and len(entry['selected_licenses']) > 0:
            nested_dict["license"] = entry['selected_licenses'][0]
        elif 'selected_licenses' in entry['mimer'] and len(entry['mimer']['selected_licenses']) > 0:
            nested_dict["license"] = entry['mimer']['selected_licenses'][0]
        elif 'licenses' in entry and len(entry['licenses']) >= 1:
            nested_dict["license"] = entry['licenses'][0]
        
        if nested_dict["license"] in license_dict.keys():
            nested_dict["obligation"] = license_dict[nested_dict["license"]]
        elif nested_dict["license"] == "MIT License":
            nested_dict["license"] = "MIT"
            nested_dict["obligation"] = license_dict["MIT"]
        
        if nested_dict["obligation"] == "unknown" or nested_dict["license"] == "unknown":
            invalid_entries.append(nested_dict)
        else:
            if foss_product_number in flat_dict.keys():
                flat_dict[foss_product_number].append(nested_dict)
            else:
                flat_dict[foss_product_number] = [nested_dict]


In [ ]:

invalid_entries      

if len(invalid_entries) > 0:
    print("Invalid entries found")
    
    import pandas as pd 
    df = pd.DataFrame(invalid_entries)
    df[["ID", "primID", "license"]].to_csv(f"unknown_licenses_{COMPONENT}.csv", index=False)

else:
    print("No invalid entries found")

In [ ]:
if len(invalid_entries) > 0:
    df.groupby("license")[["ID"]].count()



In [ ]:
from collections import OrderedDict
ordered_dict=OrderedDict(sorted(flat_dict.items()))

In [ ]:
def is_ascii(usage_text):
    return all(ord(c) < 128 for c in usage_text)
    

In [ ]:
records=[]
nonAsciiUsage = []
quotes='"'
for key,value in ordered_dict.items():
    first_entry = value[0] 
    ordered_dict = OrderedDict()
    ordered_dict["product_number"] = key
    ordered_dict["license"] = first_entry["license"]
    ordered_dict["obligation"] = first_entry["obligation"].rstrip('\n')
    usage = first_entry["usage"].strip('\"')
    ordered_dict["usage"] = f"{quotes}{usage}{quotes}"
    if not is_ascii(ordered_dict["usage"]):
        nonAsciiUsage.append(f"Non-ascii character found in usage for product number {key}", flush=True)
    records.append(ordered_dict)

records[0:10]

In [ ]:
for record in nonAsciiUsage:
    print(record)

In [ ]:
from jinja2 import Template

template = Template("""
#
# COPYRIGHT Ericsson 2022
#
#
#
# The copyright to the computer program(s) herein is the property of
#
# Ericsson Inc. The programs may be used and/or copied only with written
#
# permission from Ericsson Inc. or in accordance with the terms and
#
# conditions stipulated in the agreement/contract under which the
#
# program(s) have been supplied.
#

modelVersion: 2.0
fosses:
{%- filter indent(2) -%}
{% for record in records %}
- product_number: {{ record.product_number }}
  license: {{ record.license }}
  obligation: |
    {{ record.obligation | indent(width=4) }}
  usage: |
    {{ record.usage | indent(width=4) -}}
{% endfor %}
{%- endfilter -%}
""")


In [ ]:
output = template.render(records=records)

In [ ]:
with open(OUTPUT_FILE, 'w') as f:
    f.write(output)